In [1]:
import nltk
import threading
import queue
import feedparser
import uuid


In [2]:
threads = []

In [3]:
queues = [queue.Queue(), queue.Queue()]

In [4]:
def extractWords():
    url = 'https://timesofindia.indiatimes.com/rssfeeds/1081479906.cms'
    feed = feedparser.parse(url)
    for entry in feed['entries'][:5]:
        text = entry['title']
        if 'ex' in text:
            continue
        words = nltk.word_tokenize(text)
        data = {'uuid': uuid.uuid4(), 'input': words}
        queues[0].put(data, True)
        print(">> {} : {}".format(data['uuid'], text))

In [6]:
def extractPOS():
    while True:
        if queues[0].empty():
            break
        else:
            data = queues[0].get()
            words = data['input']
            postags = nltk.pos_tag(words)
            queues[0].task_done()
            queues[1].put({'uuid': data['uuid'], 'input': postags}, True)



In [8]:
def extractNE():
    while True:
        if queues[1].empty():
            break
        else:
            data = queues[1].get()
            postags = data['input']
            queues[1].task_done()
            chunks = nltk.ne_chunk(postags, binary=False)
            print(" << {} : ".format(data['uuid']), end = '')
            for path in chunks:
                try:
                    label = path.label()
                    print(path, end=', ')
                except:
                    pass
            print()

In [9]:
def runProgram():
    e = threading.Thread(target=extractWords())
    e.start()
    threads.append(e)
    
    p = threading.Thread(target=extractPOS())
    p.start()
    threads.append(p)

    n = threading.Thread(target=extractNE())
    n.start()
    threads.append(n)
    
    queues[0].join()
    queues[1].join()
    
    for t in threads:
        t.join()


In [10]:
if __name__ == '__main__':
    runProgram()


>> 45782902-75cf-4a05-8398-215f23ea7088 : Mahanirvan is a marvel; a sine qua non for theatre buffs
>> 73ac0bd2-3dd7-408f-9690-ef430009690f : Salman's 'Race 3' continues to impress
>> c7b38704-e848-4f08-85f2-390b311136d9 : 'Sanju' new song: A.R. Rahman's 'Ruby Ruby'
>> c7c747a1-9686-4c17-bc6c-725c913a6129 : Here's when the 'Gold' trailer will be out
>> bd52de2e-a394-4e1b-948e-20aeb025c9dc : Akshay looks intense in new 'Gold' still
 << 45782902-75cf-4a05-8398-215f23ea7088 : (GPE Mahanirvan/NNP), 
 << 73ac0bd2-3dd7-408f-9690-ef430009690f : (PERSON Salman/NNP), 
 << c7b38704-e848-4f08-85f2-390b311136d9 : (PERSON Rahman/NNP), 
 << c7c747a1-9686-4c17-bc6c-725c913a6129 : 
 << bd52de2e-a394-4e1b-948e-20aeb025c9dc : (GPE Akshay/NNP), 


In [11]:
import nltk
import threading
import queue
import feedparser
import uuid

threads = []
queues = [queue.Queue(), queue.Queue()]

def extractWords():
    url = 'https://timesofindia.indiatimes.com/rssfeeds/1081479906.cms'
    feed = feedparser.parse(url)
    for entry in feed['entries'][:5]:
        text = entry['title']
        if 'ex' in text:
            continue
        words = nltk.word_tokenize(text)
        data = {'uuid': uuid.uuid4(), 'input': words}
        queues[0].put(data, True)
        print(">> {} : {}".format(data['uuid'], text))

def extractPOS():
    while True:
        if queues[0].empty():
            break
        else:
            data = queues[0].get()
            words = data['input']
            postags = nltk.pos_tag(words)
            queues[0].task_done()
            queues[1].put({'uuid': data['uuid'], 'input': postags}, True)

def extractNE():
    while True:
        if queues[1].empty():
            break
        else:
            data = queues[1].get()
            postags = data['input']
            queues[1].task_done()
            chunks = nltk.ne_chunk(postags, binary=False)
            print("  << {} : ".format(data['uuid']), end = '')
            for path in chunks:
                try:
                    label = path.label()
                    print(path, end=', ')
                except:
                    pass
            print()

def runProgram():
    e = threading.Thread(target=extractWords())
    e.start()
    threads.append(e)

    p = threading.Thread(target=extractPOS())
    p.start()
    threads.append(p)

    n = threading.Thread(target=extractNE())
    n.start()
    threads.append(n)

    queues[0].join()
    queues[1].join()

    for t in threads:
        t.join()

if __name__ == '__main__':
    runProgram()


>> b06c11dd-7c9b-4803-b849-ce0820b76815 : Mahanirvan is a marvel; a sine qua non for theatre buffs
>> 01c5826d-a407-40e6-8fd3-8b1ecab50f0a : Salman's 'Race 3' continues to impress
>> 61aaacb4-3548-4ea5-97d8-83f4250c56cf : 'Sanju' new song: A.R. Rahman's 'Ruby Ruby'
>> 4a3e4e3c-bedd-4839-878d-dfab49d17896 : Here's when the 'Gold' trailer will be out
>> 5c551ab0-b28b-4d18-9530-92869bda7b69 : Akshay looks intense in new 'Gold' still
  << b06c11dd-7c9b-4803-b849-ce0820b76815 : (GPE Mahanirvan/NNP), 
  << 01c5826d-a407-40e6-8fd3-8b1ecab50f0a : (PERSON Salman/NNP), 
  << 61aaacb4-3548-4ea5-97d8-83f4250c56cf : (PERSON Rahman/NNP), 
  << 4a3e4e3c-bedd-4839-878d-dfab49d17896 : 
  << 5c551ab0-b28b-4d18-9530-92869bda7b69 : (GPE Akshay/NNP), 
